In [1]:
import numpy as np
import torch
from transformers import SpeechT5ForSpeechToText, SpeechT5Model, SpeechT5Config, SpeechT5PreTrainedModel

In [2]:
st5_asr = SpeechT5ForSpeechToText.from_pretrained("microsoft/speecht5_asr")

In [4]:
ckpt = torch.load("../checkpoints/speecht5_base.pt", map_location=torch.device("cpu"))

In [5]:
len(ckpt['model'].keys())

464

In [42]:
ctr1 = 1
for name, p in st5_asr.named_parameters():
    if name.startswith("speecht5.encoder.wrapped_encoder.layers.3"): #encoder.wrapped_encoder.layers.0
        print(ctr1, name, p.size())
        ctr1 += 1


1 speecht5.encoder.wrapped_encoder.layers.3.attention.k_proj.weight torch.Size([768, 768])
2 speecht5.encoder.wrapped_encoder.layers.3.attention.k_proj.bias torch.Size([768])
3 speecht5.encoder.wrapped_encoder.layers.3.attention.v_proj.weight torch.Size([768, 768])
4 speecht5.encoder.wrapped_encoder.layers.3.attention.v_proj.bias torch.Size([768])
5 speecht5.encoder.wrapped_encoder.layers.3.attention.q_proj.weight torch.Size([768, 768])
6 speecht5.encoder.wrapped_encoder.layers.3.attention.q_proj.bias torch.Size([768])
7 speecht5.encoder.wrapped_encoder.layers.3.attention.out_proj.weight torch.Size([768, 768])
8 speecht5.encoder.wrapped_encoder.layers.3.attention.out_proj.bias torch.Size([768])
9 speecht5.encoder.wrapped_encoder.layers.3.layer_norm.weight torch.Size([768])
10 speecht5.encoder.wrapped_encoder.layers.3.layer_norm.bias torch.Size([768])
11 speecht5.encoder.wrapped_encoder.layers.3.feed_forward.intermediate_dense.weight torch.Size([3072, 768])
12 speecht5.encoder.wrapped_e

In [43]:
compat = {}
ctr2 = 1
for name, p in ckpt['model'].items():
#     if name.startswith("speech_encoder_prenet"):
#         print(ctr2, name, end=" -> ")
#         parts = name.split(".")
#         tmp = name.replace("speech", "speecht5")
#         tmp = tmp.replace("_", ".", 2)
#         tmp = tmp.replace("feature_extractor", "feature_encoder")
#         tmp = tmp.replace(".0.weight", ".conv.weight")
        
#         if tmp.endswith("mask_emb"):
#             tmp = tmp.replace("mask_emb", "masked_spec_emb")
        
#         print(tmp)
#         ctr2 += 1
    if name.startswith("encoder.layers.0"):
        print(ctr2, name, p.size())
        ctr2 += 1


1 encoder.layers.0.self_attn.k_proj.weight torch.Size([768, 768])
2 encoder.layers.0.self_attn.k_proj.bias torch.Size([768])
3 encoder.layers.0.self_attn.v_proj.weight torch.Size([768, 768])
4 encoder.layers.0.self_attn.v_proj.bias torch.Size([768])
5 encoder.layers.0.self_attn.q_proj.weight torch.Size([768, 768])
6 encoder.layers.0.self_attn.q_proj.bias torch.Size([768])
7 encoder.layers.0.self_attn.out_proj.weight torch.Size([768, 768])
8 encoder.layers.0.self_attn.out_proj.bias torch.Size([768])
9 encoder.layers.0.self_attn_layer_norm.weight torch.Size([768])
10 encoder.layers.0.self_attn_layer_norm.bias torch.Size([768])
11 encoder.layers.0.fc1.weight torch.Size([3072, 768])
12 encoder.layers.0.fc1.bias torch.Size([3072])
13 encoder.layers.0.fc2.weight torch.Size([768, 3072])
14 encoder.layers.0.fc2.bias torch.Size([768])
15 encoder.layers.0.final_layer_norm.weight torch.Size([768])
16 encoder.layers.0.final_layer_norm.bias torch.Size([768])
17 encoder.layers.0.norm_k.weight torch.

In [31]:
lstm = ((2359296 * 2) + (3072 * 2) + (768 * 2)) 

In [32]:
ca = ((1179648 + 589824 + 589824) + (768 * 4) + 1536) 

In [35]:
np.asarray([lstm, ca, (lstm+ca)]) / 1e6

array([4.726272, 2.363904, 7.090176])

In [30]:
 7077888 + 12288

7090176

In [36]:
lstm/(lstm+ca)

0.6665944540727903

In [37]:
ca/(lstm+ca)

0.3334055459272097

In [36]:
st5_cfg = SpeechT5Config()

In [42]:
st5_model = SpeechT5PreTrainedModel(st5_cfg)

In [43]:
st5_model

SpeechT5PreTrainedModel()

In [32]:
st5_cfg = SpeechT5Config()

In [34]:
st5_cfg.feat_extract_norm = "layer"

In [35]:
st5_cfg

SpeechT5Config {
  "activation_dropout": 0.1,
  "apply_spec_augment": true,
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "conv_bias": false,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.1,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.1,
  "encoder_layers": 12,
  "encoder_max_relative_position": 160,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.0,
  "guided_attention_loss_num_heads": 2,
  "guided_attention_loss_scale": 10.0,
  "guided_attention_loss_sigma": 0.4,
  "hidden_act": "gelu",
  "hidden_dropout": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.

In [36]:
model = SpeechT5Model(config=st5_cfg)

In [37]:
model

SpeechT5Model(
  (encoder): SpeechT5EncoderWithoutPrenet(
    (wrapped_encoder): SpeechT5Encoder(
      (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (layers): ModuleList(
        (0-11): 12 x SpeechT5EncoderLayer(
          (attention): SpeechT5Attention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (dropout): Dropout(p=0.1, inplace=False)
          (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (feed_forward): SpeechT5FeedForward(
            (intermediate_dropout): Dropout(p=0.1, inplace=False)
            (intermediate_dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermediat

In [38]:
model.encoder.wrapped_encoder

SpeechT5Encoder(
  (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (layers): ModuleList(
    (0-11): 12 x SpeechT5EncoderLayer(
      (attention): SpeechT5Attention(
        (k_proj): Linear(in_features=768, out_features=768, bias=True)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
      )
      (dropout): Dropout(p=0.1, inplace=False)
      (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (feed_forward): SpeechT5FeedForward(
        (intermediate_dropout): Dropout(p=0.1, inplace=False)
        (intermediate_dense): Linear(in_features=768, out_features=3072, bias=True)
        (intermediate_act_fn): GELUActivation()
        (output_dense): Linear(in_features=3072, out_features=768, bias=True)
        (output_dropout): Dropout(p=0.1

In [39]:
st5_cfg_2 = SpeechT5Config()

In [40]:
model_2 = SpeechT5Model(config=st5_cfg_2)

In [41]:
model_2.encoder.wrapped_encoder

SpeechT5Encoder(
  (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (layers): ModuleList(
    (0-11): 12 x SpeechT5EncoderLayer(
      (attention): SpeechT5Attention(
        (k_proj): Linear(in_features=768, out_features=768, bias=True)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
      )
      (dropout): Dropout(p=0.1, inplace=False)
      (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (feed_forward): SpeechT5FeedForward(
        (intermediate_dropout): Dropout(p=0.1, inplace=False)
        (intermediate_dense): Linear(in_features=768, out_features=3072, bias=True)
        (intermediate_act_fn): GELUActivation()
        (output_dense): Linear(in_features=3072, out_features=768, bias=True)
        (output_dropout): Dropout(p=0.1